# T5 Poetry Generator Project
Gholamreza Dar Feb 2023

## Installing libraries

In [ ]:
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 94.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.9 MB/s eta 0:00:00


In [ ]:
!pip install -q pytorch-lightning wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.4/826.4 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 KB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 6.6 MB/s eta 0:00:00


## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Select Model

In [ ]:
mesra_mode = True # generate next mesra(True) or next beyt(False)

In [ ]:
## GHD
# model_name = "Salesforce/codet5-small"
model_name = "google/mt5-small"

## Download dataset

In [ ]:
## GHD
from datasets import load_dataset
hafez_dataset = load_dataset("Gholamreza/hafez")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/Gholamreza___csv/Gholamreza--hafez-7de6f4c44843e903/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
## GHD
hafez_dataset

DatasetDict({
    train: Dataset({
        features: ['cat_title', 'v_order', 'v_position', 'poem_text', 'poet_name', 'poem_id'],
        num_rows: 8610
    })
    test: Dataset({
        features: ['cat_title', 'v_order', 'v_position', 'poem_text', 'poet_name', 'poem_id'],
        num_rows: 1000
    })
})

<!-- ## Preprocess data

Here, we load the "code_to_text" portion of the [CodeXGLUE](https://microsoft.github.io/CodeXGLUE/) dataset. As you may know, GLUE [(Wang et al., 2018)](https://arxiv.org/abs/1804.07461) is a famous benchmark in NLP, which led to a lot of progress (see the leaderboard [here](https://gluebenchmark.com/)). Microsoft has now created a similar benchmark called CodeXGLUE [(Lu et al., 2021)](https://arxiv.org/abs/2102.04664), but for code + natural language instead of just natural language. It consits of several subtasks (similar to GLUE).

Let's only load the examples of the Ruby programming language. This is a fairly small dataset, which is ideally suited for demonstration purposes in Google Colab. The Python split has way more training examples (250,000), but training this in Google Colab isn't ideal. -->

### Split dataset (input/label)

In [ ]:
## GHD
def split_dataset(dataset, mesra_mode=True, split="train"):
  '''This function takes in the hafez dataset, selects mesras or beyts as inputs and labels(outputs) in an alternating fashion.'''

  if not mesra_mode:
    prefix = "Generate beyt: "
    inputs = []
    labels = []
    sep = "<sep>"

    for i in range(len(dataset[split])//2):
      mesra_1 = dataset[split][i]
      mesra_2 = dataset[split][i+1]
      if i%2==0:
        inputs.append(prefix + mesra_1["poem_text"] + sep + mesra_2["poem_text"])
      else:
        labels.append(mesra_1["poem_text"] + sep + mesra_2["poem_text"])
    
    return inputs, labels
  
  if mesra_mode:
    prefix = "Generate mesra: "
    inputs = []
    labels = []

    for i, mesra in enumerate(dataset[split]):
      if i%2==0:
        inputs.append(prefix + mesra["poem_text"])
      else:
        labels.append(mesra["poem_text"])
    
    return inputs, labels



In [ ]:
inputs, labels = split_dataset(hafez_dataset, split="train")

In [ ]:
## GHD
inputs[:5]

['Generate mesra: جز نقش تو در نظر نیامد ما را',
 'Generate mesra: خواب ارچه خوش آمد همه را در عهدت',
 'Generate mesra: بر گیر شراب طرب\u200cانگیز و بیا',
 'Generate mesra: مشنو سخن خصم که بنشین و مرو',
 'Generate mesra: ماهی که قدش به سرو می\u200cماند راست']

In [ ]:
 ## GHD
 labels[:5]

['جز کوی تو رهگذر نیامد ما را',
 'حقا که به چشم در نیامد ما را',
 'پنهان ز رقیب سفله بستیز و بیا',
 'بشنو ز من این نکته که برخیز و بیا',
 'آیینه به دست و روی خود می\u200cآراست']

In [ ]:
from datasets import Dataset

# Train dataset
inputs, labels = split_dataset(hafez_dataset, split="train")
hafez_dict_train = {"input":inputs, "label":labels}
hafez_dataset_train = Dataset.from_dict(hafez_dict_train)

# Test dataset
inputs, labels = split_dataset(hafez_dataset, split="test")
hafez_dict_test = {"input":inputs, "label":labels}
hafez_dataset_test = Dataset.from_dict(hafez_dict_test)

In [ ]:
del hafez_dataset

In [ ]:
from datasets import DatasetDict
hafez_datasets = DatasetDict({"train":hafez_dataset_train, "test":hafez_dataset_test})

In [ ]:
hafez_datasets

DatasetDict({
    train: Dataset({
        features: ['input', 'label'],
        num_rows: 4305
    })
    test: Dataset({
        features: ['input', 'label'],
        num_rows: 500
    })
})

In [ ]:
## GHD
!pip install sentencepiece 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Tokenize dataset

In [ ]:
## GHD

from transformers import RobertaTokenizer, AutoTokenizer

tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-small") #? 
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

max_input_length = 256
max_target_length = 128

def preprocess_examples(examples):
  input_list = examples['input']
  label_list = examples['label']
  
  # encode the inputs(mesra 1 for example)
  model_inputs = tokenizer(input_list, max_length=max_input_length, padding="max_length", truncation=True)

  # encode the labels(mesra 2 for example)
  labels = tokenizer(label_list, max_length=max_target_length, padding="max_length", truncation=True).input_ids

  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)
  
  model_inputs["labels"] = labels_with_ignore_index

  return model_inputs

Now that we have defined the function, let's call `.map()` on the HuggingFace Dataset object, which allows us to apply this function in batches (by default a batch size of 1,000 is used!) - hence super fast.

In [ ]:
dataset = hafez_datasets.map(preprocess_examples, batched=True, remove_columns=hafez_datasets["train"].column_names)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4305
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 500
    })
})

Next, let's set the format to "torch" and create PyTorch dataloaders.

### Train Valid Test split

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch

dataset.set_format(type="torch", columns=['input_ids', 'attention_mask', 'labels'])

train_dataset, valid_dataset = random_split(dataset["train"], [0.9, 0.1], generator=torch.Generator().manual_seed(46))

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)
valid_dataloader = DataLoader(valid_dataset, batch_size=4)
test_dataloader = DataLoader(dataset['test'], batch_size=4)

In [ ]:
batch = next(iter(train_dataloader))
print(batch.keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


### Example tokenized data

In [ ]:
batch['input_ids'][0]

tensor([162412,    265,   4375,    571,    267,  22959,    406,  45669,   7773,
           406,  16928,  25538,  72292,      1,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [ ]:
tokenizer.decode(batch['input_ids'][0])

'Generate mesra: سلامی چو بوی خوش آشنایی</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

In [ ]:
labels = batch['labels'][0]
tokenizer.decode([label for label in labels if label != -100])

'بدان مردم دیده روشنایی</s>'

<!-- ## Fine-tune using PyTorch Lightning

As we will train the model using PyTorch Lightning, we first need to define a `LightningModule`, which is an `nn.Module` with some additional functionalities. We just need to define the `forward` pass, `training_step` (and optionally `validation_step` and `test_step`), and the corresponding dataloaders. PyTorch Lightning will then automate the training for us, handling device placement (i.e. we don't need to type `.to(device)` anywhere), etc. It also comes with support for loggers (such as Tensorboard, Weights and Biases) and callbacks.

Of course, you could also train the model in other ways:
* using regular PyTorch
* using the HuggingFace Trainer (in this case, the Seq2SeqTrainer)
* using HuggingFace Accelerate
* etc. -->

## Training

In [ ]:
from transformers import MT5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
import pytorch_lightning as pl

class PoemMT5(pl.LightningModule):
    def __init__(self, lr=5e-5, num_train_epochs=15, warmup_steps=1000):
        super().__init__()
        self.model = MT5ForConditionalGeneration.from_pretrained(model_name) # GHD
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels=None):     
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs
    
    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss
      
    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss, on_epoch=True)
        # print("loss", loss)

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)     

        return loss

    def configure_optimizers(self):
        # create optimizer
        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)
        # create learning rate scheduler
        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {'scheduler': get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.hparams.warmup_steps,
                                                    num_training_steps=num_train_optimization_steps),
                        'name': 'learning_rate',
                        'interval':'step',
                        'frequency': 1}
        
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}
      
    def training_epoch_end(self, training_step_outputs):
        pass

        # print("End of epoch results")
        # for i in range(5):
        #     example = hafez_datasets["test"][i]
        #     input_ids = tokenizer(example['input'], return_tensors='pt').input_ids
        #     outputs = self.model.generate(input_ids.cuda(), num_beams=3).cpu().detach().numpy()
        #     print(f"{i})", example["input"])
        #     print("Next Mesra:", tokenizer.decode(outputs[0], skip_special_tokens=True))
        #     print("Real Mesra:", example["label"])
        #     print()
        # print()


    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader

In [ ]:
model = PoemMT5()

In [ ]:
# model.model = model.model.from_pretrained(save_directory)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# del model

In [ ]:
!nvidia-smi

Thu Feb 23 21:39:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## ||| Training

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

early_stop_callback = EarlyStopping(
    monitor='validation_loss',
    patience=3,
    strict=False,
    verbose=False,
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = Trainer(accelerator='gpu', 
                  # limit_train_batches=0.15,
                  default_root_dir="/content/Checkpoints",
                  callbacks=[early_stop_callback, lr_monitor])
trainer.fit(model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  |

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


Once we're done training, we can also save the HuggingFace model as follows:

In [ ]:
save_directory="." # save in the current working directory, you can change this of course
model.model.save_pretrained(save_directory)

In [ ]:
# save to drive
!cp pytorch_model.bin /content/gdrive/MyDrive/NLU_Exercises/pytorch_model.bin

This allows us to easily load the trained model again using the `from_pretrained()` method, as shown below.

## Inference

Now that we've trained a model, let's test it on some examples from the test set.

In [ ]:
from transformers import MT5ForConditionalGeneration

model = MT5ForConditionalGeneration.from_pretrained(save_directory)

### Prediction

In [ ]:
## GHD

example = hafez_datasets["test"][1]
example

{'input': 'Generate mesra: گفتم سخن تو، گفت حافظ گفتا',
 'label': 'شادی همه لطیفه گویان صلوات'}

In [ ]:
input_ids = tokenizer(example['input'], return_tensors='pt').input_ids
input_ids

tensor([[162412,    265,   4375,    571,    267,   5021,    633,  19220,    586,
           2027,    343,   5021,    259,  34258,   5021,    632,      1]])

In [ ]:
outputs = model.generate(input_ids, num_return_sequences=5, num_beams=5)
outputs

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[     0, 250099,    633,  19220,    586,   2027,    343,   5021,    633,
          19220,    586,   2027,    343,   5021,    633,  19220,    586,   2027,
            343,   5021],
        [     0, 250099,    633,  19220,    586,   2027,    343,   5021,    633,
          19220,    586,   2027,    343,   5021,    633,  19220,    586,   2027,
           5021,    633],
        [     0, 250099,    633,  19220,    586,   2027,    343,   5021,    633,
          19220,    586,   2027,    343,   5021,    633,  19220,    586,   2027,
            343,  19220],
        [     0, 250099,    633,  19220,    586,   2027,    343,   5021,    633,
          19220,    586,   2027,    343,   5021,    633,  19220,    586,   2027,
            259,    343],
        [     0, 250099,    633,  19220,    586,   2027,    343,   5021,    633,
          19220,    586,   2027,    343,   5021,    633,  19220,    586,   2027,
            343,      1]])

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=False)

'<pad><extra_id_0>م سخن تو، گفتم سخن تو، گفتم سخن تو، گفت'

In [ ]:
input_ids = tokenizer(example['input'], return_tensors='pt').input_ids
outputs = model.generate(input_ids)
print(">", example["input"])
print("Next Mesra:", tokenizer.decode(outputs[0], skip_special_tokens=True))

> Generate mesra: گفتم سخن تو، گفت حافظ گفتا
Next Mesra: <extra_id_0> تو


### Interactive generation 

In [ ]:
input_text = "سلام ای ساقی خوبان"

In [ ]:
prefix = "Generate Mesra: " if mesra_mode else "Generate Beyt: "
input_ids = tokenizer(prefix + input_text, return_tensors='pt').input_ids
outputs = model.generate(input_ids)
print(">", example["input"])
print("Next Mesra:", tokenizer.decode(outputs[0], skip_special_tokens=True))

> Generate mesra: گفتم سخن تو، گفت حافظ گفتا
Next Mesra: <extra_id_0> به


### Generate next for test dataset

In [ ]:
## GHD

for i in range(len(hafez_datasets["test"])):
  example = hafez_datasets["test"][i]
  input_ids = tokenizer(example['input'], return_tensors='pt').input_ids
  outputs = model.generate(input_ids, num_beams=3)
  print(f"{i})", example["input"])
  print("Next Mesra:", tokenizer.decode(outputs[0], skip_special_tokens=True))
  print("Real Mesra:", example["label"])
  print()

0) Generate mesra: گفتم که لبت، گفت لبم آب حیات
Next Mesra: <extra_id_0> به
Real Mesra: گفتم دهنت، گفت زهی حب نبات

1) Generate mesra: گفتم سخن تو، گفت حافظ گفتا
Next Mesra: <extra_id_0> تو
Real Mesra: شادی همه لطیفه گویان صلوات

2) Generate mesra: ای کاش که بخت سازگاری کردی
Next Mesra: <extra_id_0> کرد
Real Mesra: با جور زمانه یار یاری کردی

3) Generate mesra: از دست جوانی‌ام چو بربود عنان
Next Mesra: <extra_id_0> به
Real Mesra: پیری چو رکاب پایداری کردی

4) Generate mesra: دادگرا تو را فلک جرعه کش پیاله باد
Next Mesra: <extra_id_0> را فلک
Real Mesra: دشمن دل سیاه تو غرقه به خون چو لاله باد

5) Generate mesra: ذروه‌ی کاخ رتبتت راست ز فرط ارتفاع
Next Mesra: <extra_id_0> کاخ
Real Mesra: راهروان وهم را راه هزار ساله باد

6) Generate mesra: ای مه برج منزلت چشم و چراغ عالمی
Next Mesra: <extra_id_0> چشم و چراغ عالمی
Real Mesra: باده‌ی صاف دایمت در قدح و پیاله باد

7) Generate mesra: چون به هوای مدحتت زهره شود ترانه‌ساز
Next Mesra: <extra_id_0> زهره شود
Real Mesra: حاسدت از سماع آن محروم آه 

KeyboardInterrupt: ignored

## Tensorboard

In [ ]:
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir="/content/Checkpoints/lightning_logs/version_1"